In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, RepeatedKFold
from scipy import sparse
# Show all columns
pd.set_option('display.max_columns', None)
# Show all rows
pd.set_option('display.max_rows', None)
from datetime import datetime

In [5]:
# Import data
train_abbr=pd.read_csv("happiness_train_abbr.csv",encoding='ISO-8859-1')
train=pd.read_csv("happiness_train_complete.csv",encoding='ISO-8859-1')
test_abbr=pd.read_csv("happiness_test_abbr.csv",encoding='ISO-8859-1')
test=pd.read_csv("happiness_test_complete.csv",encoding='ISO-8859-1')
test_sub=pd.read_csv("happiness_submit.csv",encoding='ISO-8859-1')

In [6]:
# Observe data
print(test.shape,test_sub.shape,train.shape)

(2968, 139) (2968, 2) (8000, 140)


In [7]:
train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,religion,religion_freq,edu,edu_other,edu_status,edu_yr,income,political,join_party,floor_area,property_0,property_1,property_2,property_3,property_4,property_5,property_6,property_7,property_8,property_other,height_cm,weight_jin,health,health_problem,depression,hukou,hukou_loc,media_1,media_2,media_3,media_4,media_5,media_6,leisure_1,leisure_2,leisure_3,leisure_4,leisure_5,leisure_6,leisure_7,leisure_8,leisure_9,leisure_10,leisure_11,leisure_12,socialize,relax,learn,social_neighbor,social_friend,socia_outing,equity,class,class_10_before,class_10_after,class_14,work_exper,work_status,work_yr,work_type,work_manage,insur_1,insur_2,insur_3,insur_4,family_income,family_m,family_status,house,car,invest_0,invest_1,invest_2,invest_3,invest_4,invest_5,invest_6,invest_7,invest_8,invest_other,son,daughter,minor_child,marital,marital_1st,s_birth,marital_now,s_edu,s_political,s_hukou,s_income,s_work_exper,s_work_status,s_work_type,f_birth,f_edu,f_political,f_work_14,m_birth,m_edu,m_political,m_work_14,status_peer,status_3_before,view,inc_ability,inc_exp,trust_1,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,trust_10,trust_11,trust_12,trust_13,neighbor_familiarity,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,1,1,11,NaN,4.0,-2.0,20000,1,NaN,45.0,0,1,0,0,0,0,0,0,0,NaN,176,155,3,2,5,5,2.0,4,2,5,5,4,3,1,4,3,1,2,3,4,1,4,5,4,1,2,4,3,3.0,3.0,2,3,3,3,3,1,1,3.0,30.0,1.0,2.0,1,1,1,2,60000.0,2,2,1,2,0,1,0,0,0,0,0,0,0,NaN,1,0,0.0,3,1984.0,1958.0,1984.0,6.0,1.0,5.0,40000.0,5.0,NaN,NaN,-2,4,4,1,-2,4,1,1,3,2,4,3,50000.0,4,2,-8,-8,5,3,2,3,4,3,-8,4,1,4,50,60,50,50,30.0,30,50,50,50
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,1,1,12,NaN,4.0,2013.0,20000,1,NaN,110.0,0,0,0,0,1,0,0,0,0,NaN,170,110,5,4,3,1,1.0,2,2,1,3,5,1,2,3,4,3,5,4,3,2,3,4,5,1,2,4,3,6.0,2.0,1,3,6,4,8,5,1,3.0,2.0,1.0,3.0,1,1,1,1,40000.0,3,4,1,2,0,1,0,0,0,0,0,0,0,NaN,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1972,3,1,2,1973,3,1,2,1,1,4,2,50000.0,5,4,4,3,5,3,3,3,2,3,3,3,2,3,90,70,70,80,85.0,70,90,60,60
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,0,3,4,NaN,4.0,-2.0,2000,1,NaN,120.0,0,1,1,0,0,0,0,0,0,NaN,160,122,4,4,5,1,1.0,2,2,2,5,1,3,1,4,4,3,5,4,4,2,3,5,5,5,3,4,2,2.0,5.0,2,4,5,4,6,3,2,NaN,NaN,NaN,NaN,1,1,2,2,8000.0,3,3,1,2,0,1,0,0,0,0,0,0,0,NaN,0,2,1.0,3,1990.0,1968.0,1990.0,3.0,1.0,1.0,6000.0,3.0,NaN,NaN,-2,1,1,2,-2,1,1,2,2,1,4,2,80000.0,3,3,3,3,4,3,3,3,3,3,-8,3,1,4,90,80,75,79,80.0,90,90,90,75
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,1,1,3,NaN,4.0,1959.0,6420,1,NaN,78.0,0,0,0,1,0,0,0,0,0,NaN,163,170,4,4,4,1,2.0,2,1,1,5,1,1,1,5,2,4,5,4,5,1,1,5,5,5,2,4,4,1.0,6.0,1,4,5,5,7,2,4,NaN,NaN,NaN,NaN,2,2,2,2,12000.0,3,3,1,1,0,1,0,0,0,0,0,0,0,NaN,1,4,0.0,7,1960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2,14,1,2,-2,1,1,2,2,1,3,2,10000.0,3,3,4,3,5,3,3,5,4,3,3,3,2,3,100,90,70,80,80.0,90,90,80,80
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,1,1,12,NaN,1.0,2014.0,-1,2,NaN,70.0,0,0,0,0,1,0,0,0,0,NaN,165,110,5,5,3,2,3.0,1,3,4,2,5,5,3,3,3,2,4,4,3,5,2,5,5,1,4,3,4,7.0,5.0,3,2,1,1,1,4,6,NaN,NaN,NaN,NaN,1,2,2,2,-2.0,4,3,1,1,0,1,0,0,0,0,0,0,0,NaN,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970,6,1,10,1972,4,1,15,3,2,3,-8,200000.0,4,3,3,3,5,5,3,4,3,3,3,3,2,2,50,50,50,50,50.0,50,50,50,50


In [8]:
# Check missing data
train.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 140 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    8000 non-null   int64  
 1   happiness             8000 non-null   int64  
 2   survey_type           8000 non-null   int64  
 3   province              8000 non-null   int64  
 4   city                  8000 non-null   int64  
 5   county                8000 non-null   int64  
 6   survey_time           8000 non-null   object 
 7   gender                8000 non-null   int64  
 8   birth                 8000 non-null   int64  
 9   nationality           8000 non-null   int64  
 10  religion              8000 non-null   int64  
 11  religion_freq         8000 non-null   int64  
 12  edu                   8000 non-null   int64  
 13  edu_other             3 non-null      object 
 14  edu_status            6880 non-null   float64
 15  edu_yr              

In [9]:
# View label distribution
y_train_=train["happiness"]
y_train_.value_counts()

 4    4818
 5    1410
 3    1159
 2     497
 1     104
-8      12
Name: happiness, dtype: int64

In [10]:
# Replace -8 with 3
y_train_=y_train_.map(lambda x:3 if x==-8 else x)

In [11]:
# Let label start from 0
y_train_=y_train_.map(lambda x:x-1)

In [12]:
# Concatenate 'train' and 'test' together
data = pd.concat([train,test],axis=0,ignore_index=True)
data.shape

(10968, 140)

In [13]:
# Processing time features
data['survey_time'] = pd.to_datetime(data['survey_time'],format='%Y-%m-%d %H:%M:%S')
data["weekday"]=data["survey_time"].dt.weekday
data["year"]=data["survey_time"].dt.year
data["quarter"]=data["survey_time"].dt.quarter
data["hour"]=data["survey_time"].dt.hour
data["month"]=data["survey_time"].dt.month

In [14]:
# Divide the time of day
def hour_cut(x):
    if 0<=x<6:
        return 0
    elif  6<=x<8:
        return 1
    elif  8<=x<12:
        return 2
    elif  12<=x<14:
        return 3
    elif  14<=x<18:
        return 4
    elif  18<=x<21:
        return 5
    elif  21<=x<24:
        return 6
        
data["hour_cut"]=data["hour"].map(hour_cut)

In [15]:
# Extract age features
data["survey_age"]=data["year"]-data["birth"]

In [16]:
# Let label start from 0
data["happiness"]=data["happiness"].map(lambda x:x-1)

In [17]:
# Remove three features with many missing values
data=data.drop(["edu_other"], axis=1)
data=data.drop(["happiness"], axis=1)
data=data.drop(["survey_time"], axis=1)

In [18]:
# Whether join the party
data["join_party"]=data["join_party"].map(lambda x:0 if pd.isnull(x)  else 1)

In [19]:
# We discretized some variables in order to ensure stability
# era of birth
def birth_split(x):
    if 1920<=x<=1930:
        return 0
    elif  1930<x<=1940:
        return 1
    elif  1940<x<=1950:
        return 2
    elif  1950<x<=1960:
        return 3
    elif  1960<x<=1970:
        return 4
    elif  1970<x<=1980:
        return 5
    elif  1980<x<=1990:
        return 6
    elif  1990<x<=2000:
        return 7
    
data["birth_s"]=data["birth"].map(birth_split)
   

In [20]:
# Income grouping
def income_cut(x):
    if x<0:
        return 0
    elif  0<=x<1200:
        return 1
    elif  1200<x<=10000:
        return 2
    elif  10000<x<24000:
        return 3
    elif  24000<x<40000:
        return 4
    elif  40000<=x:
        return 5
 

    
data["income_cut"]=data["income"].map(income_cut)

In [21]:
# Data input
data["edu_status"]=data["edu_status"].fillna(5)
data["edu_yr"]=data["edu_yr"].fillna(-2)
data["property_other"]=data["property_other"].map(lambda x:0 if pd.isnull(x)  else 1)
data["hukou_loc"]=data["hukou_loc"].fillna(1)
data["social_neighbor"]=data["social_neighbor"].fillna(8)
data["social_friend"]=data["social_friend"].fillna(8)
data["work_status"]=data["work_status"].fillna(0)
data["work_yr"]=data["work_yr"].fillna(0)
data["work_type"]=data["work_type"].fillna(0)
data["work_manage"]=data["work_manage"].fillna(0)
data["family_income"]=data["family_income"].fillna(-2)
data["invest_other"]=data["invest_other"].map(lambda x:0 if pd.isnull(x)  else 1)
data["minor_child"]=data["minor_child"].fillna(0)
data["marital_1st"]=data["marital_1st"].fillna(0)
data["s_birth"]=data["s_birth"].fillna(0)
data["marital_now"]=data["marital_now"].fillna(0)
data["s_edu"]=data["s_edu"].fillna(0)
data["s_political"]=data["s_political"].fillna(0)
data["s_hukou"]=data["s_hukou"].fillna(0)
data["s_income"]=data["s_income"].fillna(0)
data["s_work_exper"]=data["s_work_exper"].fillna(0)
data["s_work_status"]=data["s_work_status"].fillna(0)
data["s_work_type"]=data["s_work_type"].fillna(0)

In [22]:
data=data.drop(["id"], axis=1)

In [23]:
X_train_ = data[:train.shape[0]]
X_test_  = data[train.shape[0]:]

In [24]:
target_column = 'happiness'
feature_columns=list(X_test_.columns) 
feature_columns

['survey_type',
 'province',
 'city',
 'county',
 'gender',
 'birth',
 'nationality',
 'religion',
 'religion_freq',
 'edu',
 'edu_status',
 'edu_yr',
 'income',
 'political',
 'join_party',
 'floor_area',
 'property_0',
 'property_1',
 'property_2',
 'property_3',
 'property_4',
 'property_5',
 'property_6',
 'property_7',
 'property_8',
 'property_other',
 'height_cm',
 'weight_jin',
 'health',
 'health_problem',
 'depression',
 'hukou',
 'hukou_loc',
 'media_1',
 'media_2',
 'media_3',
 'media_4',
 'media_5',
 'media_6',
 'leisure_1',
 'leisure_2',
 'leisure_3',
 'leisure_4',
 'leisure_5',
 'leisure_6',
 'leisure_7',
 'leisure_8',
 'leisure_9',
 'leisure_10',
 'leisure_11',
 'leisure_12',
 'socialize',
 'relax',
 'learn',
 'social_neighbor',
 'social_friend',
 'socia_outing',
 'equity',
 'class',
 'class_10_before',
 'class_10_after',
 'class_14',
 'work_exper',
 'work_status',
 'work_yr',
 'work_type',
 'work_manage',
 'insur_1',
 'insur_2',
 'insur_3',
 'insur_4',
 'family_income'

In [25]:
X_train = np.array(X_train_)
y_train = np.array(y_train_)
X_test  = np.array(X_test_)

In [26]:
print(X_train.shape, y_train.shape, X_test.shape)

(8000, 145) (8000,) (2968, 145)


In [27]:
# Customize evaluation function
def myFeval(preds, xgbtrain):
    label = xgbtrain.get_label()
    score = mean_squared_error(label,preds)
    return 'myFeval',score

In [28]:
##### xgb

xgb_params = {"booster":'gbtree','eta': 0.005, 'max_depth': 5, 'subsample': 0.7, 
              'colsample_bytree': 0.8, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 8}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])
    
    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params,feval = myFeval)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, y_train_)))

fold n°1
[12:42:14] WARNING: D:\Build\xgboost\xgboost-1.1.0.git\python-package\build\temp.win-amd64-3.7\Release\xgboost\src\objective\regression_obj.cu:168: reg:linear is now deprecated in favor of reg:squarederror.
[12:42:14] WARNING: D:\Build\xgboost\xgboost-1.1.0.git\python-package\build\temp.win-amd64-3.7\Release\xgboost\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:2.49564	valid_data-rmse:2.48131	train-myFeval:6.22821	valid_data-myFeval:6.15691
Multiple eval metrics have been passed: 'valid_data-myFeval' will be used for early stopping.

Will train until valid_data-myFeval hasn't improved in 200 rounds.
[100]	train-rmse:1.61303	valid_data-rmse:1.60278	train-myFeval:2.60185	valid_data-myFeval:2.56889
[200]	train-rm

Multiple eval metrics have been passed: 'valid_data-myFeval' will be used for early stopping.

Will train until valid_data-myFeval hasn't improved in 200 rounds.
[100]	train-rmse:1.60726	valid_data-rmse:1.63461	train-myFeval:2.58328	valid_data-myFeval:2.67196
[200]	train-rmse:1.10986	valid_data-rmse:1.14806	train-myFeval:1.23178	valid_data-myFeval:1.31804
[300]	train-rmse:0.84606	valid_data-rmse:0.89805	train-myFeval:0.71582	valid_data-myFeval:0.80650
[400]	train-rmse:0.71539	valid_data-rmse:0.78128	train-myFeval:0.51178	valid_data-myFeval:0.61040
[500]	train-rmse:0.65159	valid_data-rmse:0.72989	train-myFeval:0.42457	valid_data-myFeval:0.53274
[600]	train-rmse:0.61847	valid_data-rmse:0.70757	train-myFeval:0.38251	valid_data-myFeval:0.50066
[700]	train-rmse:0.59862	valid_data-rmse:0.69739	train-myFeval:0.35835	valid_data-myFeval:0.48635
[800]	train-rmse:0.58472	valid_data-rmse:0.69223	train-myFeval:0.34190	valid_data-myFeval:0.47918
[900]	train-rmse:0.57363	valid_data-rmse:0.68916	train

[1300]	train-rmse:0.53611	valid_data-rmse:0.68128	train-myFeval:0.28741	valid_data-myFeval:0.46414
[1400]	train-rmse:0.52822	valid_data-rmse:0.68074	train-myFeval:0.27902	valid_data-myFeval:0.46341
[1500]	train-rmse:0.52061	valid_data-rmse:0.68003	train-myFeval:0.27103	valid_data-myFeval:0.46244
[1600]	train-rmse:0.51375	valid_data-rmse:0.67974	train-myFeval:0.26394	valid_data-myFeval:0.46204
[1700]	train-rmse:0.50674	valid_data-rmse:0.67919	train-myFeval:0.25679	valid_data-myFeval:0.46130
[1800]	train-rmse:0.50024	valid_data-rmse:0.67881	train-myFeval:0.25024	valid_data-myFeval:0.46079
[1900]	train-rmse:0.49373	valid_data-rmse:0.67870	train-myFeval:0.24377	valid_data-myFeval:0.46064
[2000]	train-rmse:0.48750	valid_data-rmse:0.67855	train-myFeval:0.23765	valid_data-myFeval:0.46043
[2100]	train-rmse:0.48135	valid_data-rmse:0.67817	train-myFeval:0.23169	valid_data-myFeval:0.45992
[2200]	train-rmse:0.47558	valid_data-rmse:0.67815	train-myFeval:0.22618	valid_data-myFeval:0.45988
[2300]	tra

In [29]:
##### lgb 

param = {'boosting_type': 'gbdt',
         'num_leaves': 20,
         'min_data_in_leaf': 20, 
         'objective':'regression',
         'max_depth':6,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(len(X_train_))
predictions_lgb = np.zeros(len(X_test_))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
   # print(trn_idx)
   # print(".............x_train.........")
   # print(X_train[trn_idx])
  #  print(".............y_train.........")
  #  print(y_train[trn_idx])
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])
    
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, y_train_)))

fold n°1
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.437503	valid_1's l2: 0.469686
[400]	training's l2: 0.372168	valid_1's l2: 0.44976
[600]	training's l2: 0.33182	valid_1's l2: 0.443816
[800]	training's l2: 0.300597	valid_1's l2: 0.4413
Early stopping, best iteration is:
[852]	training's l2: 0.293435	valid_1's l2: 0.440755
fold n°2
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.431328	valid_1's l2: 0.494627
[400]	training's l2: 0.366744	valid_1's l2: 0.46981
[600]	training's l2: 0.327688	valid_1's l2: 0.463121
[800]	training's l2: 0.297368	valid_1's l2: 0.459899
[1000]	training's l2: 0.272359	valid_1's l2: 0.458902
[1200]	training's l2: 0.251022	valid_1's l2: 0.457813
Early stopping, best iteration is:
[1175]	training's l2: 0.253627	valid_1's l2: 0.457521
fold n°3
Training until validation scores don't improve for 100 rounds
[200]	training's l2: 0.429379	valid_1's l2: 0.499227
[400]	training's l2: 0.3656	va

In [30]:
#!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple catboost

In [31]:
from catboost import Pool, CatBoostRegressor
# cat_features=[0,2,3,10,11,13,15,16,17,18,19]
from sklearn.model_selection import train_test_split


#X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train_, y_train_, test_size=0.3, random_state=2019)
# train_pool = Pool(X_train_s, y_train_s,cat_features=[0,2,3,10,11,13,15,16,17,18,19])
# val_pool = Pool(X_test_s, y_test_s,cat_features=[0,2,3,10,11,13,15,16,17,18,19])
# test_pool = Pool(X_test_ ,cat_features=[0,2,3,10,11,13,15,16,17,18,19]) 


kfolder = KFold(n_splits=5, shuffle=True, random_state=2019)
oof_cb = np.zeros(len(X_train_))
predictions_cb = np.zeros(len(X_test_))
kfold = kfolder.split(X_train_, y_train_)
fold_=0
#X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train, y_train, test_size=0.3, random_state=2019)
for train_index, vali_index in kfold:
    print("fold n°{}".format(fold_))
    fold_=fold_+1
    k_x_train = X_train[train_index]
    k_y_train = y_train[train_index]
    k_x_vali = X_train[vali_index]
    k_y_vali = y_train[vali_index]
    cb_params = {
         'n_estimators': 100000,
         'loss_function': 'RMSE',
         'eval_metric':'RMSE',
         'learning_rate': 0.05,
         'depth': 5,
         'use_best_model': True,
         'subsample': 0.6,
         'bootstrap_type': 'Bernoulli',
         'reg_lambda': 3
    }
    model_cb = CatBoostRegressor(**cb_params)
    #train the model
    model_cb.fit(k_x_train, k_y_train,eval_set=[(k_x_vali, k_y_vali)],verbose=100,early_stopping_rounds=50)
    oof_cb[vali_index] = model_cb.predict(k_x_vali, ntree_end=model_cb.best_iteration_)
    predictions_cb += model_cb.predict(X_test_, ntree_end=model_cb.best_iteration_) / kfolder.n_splits



print("CV score: {:<8.8f}".format(mean_squared_error(oof_cb, y_train_)))


fold n°0
0:	learn: 0.8175871	test: 0.7820939	best: 0.7820939 (0)	total: 70.5ms	remaining: 1h 57m 29s
100:	learn: 0.6711041	test: 0.6749289	best: 0.6749289 (100)	total: 1.47s	remaining: 24m 13s
200:	learn: 0.6410910	test: 0.6688829	best: 0.6686703 (190)	total: 2.65s	remaining: 21m 55s
300:	learn: 0.6130819	test: 0.6669464	best: 0.6668201 (282)	total: 3.89s	remaining: 21m 27s
400:	learn: 0.5895197	test: 0.6666901	best: 0.6663658 (371)	total: 5.05s	remaining: 20m 53s
500:	learn: 0.5684832	test: 0.6657841	best: 0.6654600 (478)	total: 6.24s	remaining: 20m 39s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6654599993
bestIteration = 478

Shrink model to first 479 iterations.
fold n°1
0:	learn: 0.8107754	test: 0.8172376	best: 0.8172376 (0)	total: 13.8ms	remaining: 23m 3s
100:	learn: 0.6715406	test: 0.6800052	best: 0.6800052 (100)	total: 1.21s	remaining: 19m 52s
200:	learn: 0.6428284	test: 0.6699391	best: 0.6699391 (200)	total: 2.44s	remaining: 20m 11s
300:	learn: 0.614450

In [32]:
from sklearn import linear_model
# Stacking the results of lgb, xgb and ctb
train_stack = np.vstack([oof_lgb,oof_xgb,oof_cb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb,predictions_cb]).transpose()


folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2018)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,y_train)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], y_train[trn_idx]
    val_data, val_y = train_stack[val_idx], y_train[val_idx]
    
    clf_3 = linear_model.BayesianRidge()
    #clf_3 =linear_model.Ridge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_stack, y_train_)))


fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
CV score: 0.45321528


In [33]:
result=list(predictions)
result=list(map(lambda x: x + 1, result))
test_sub["happiness"]=result
test_sub.to_csv("happiness_20200728.csv", index=False)